# Novo - Accumulations


In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("Offenlegen_NEU_Liste v2_FNALE LISTE_NEU_NEU_AKOL-080818.pdf", pages='all', lattice=True)

## Format Table

In [3]:
df_export = df.copy()

#Shift all where column[0]  empty
column0 = df_export[df_export.columns[0]]
df_export[column0.isna()] = df_export[column0.isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[[7]],axis=1)
df_export = df_export.drop(df.columns[9:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of', na=False)]
rnd = df_export[df_export.type.str.contains(' Research & Development', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Fix RnD
df_export.loc[df_export.type == 'rnd', 'total'] = df_export.loc[df_export.type == 'rnd', 'sponsorship']
df_export.loc[df_export.type == 'rnd', 'donations_grants'] = np.NaN
df_export.loc[df_export.type == 'rnd', 'sponsorship'] = np.NaN

#Numberize and sum
df_export = cleanup_number(df_export)
df_export = remove_in_numbers(df_export, '-')
df_export = amounts_to_number(df_export)

export_acumulations(df_export, 'novo')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')